In [1]:
from phase2_module.data.load import Data
import warnings
import pandas as pd
warnings.filterwarnings(action='ignore')


train = Data.train

In [2]:
def set_up(mode= None):
    pd.set_option('display.max_rows', mode)
# pd.options.display.max_rows = 60
set_up()

In [3]:
train["inquiry_type"].isnull().sum()

941

In [4]:
train["inquiry_type"].fillna("-", inplace=True)

## Inquiry Type 전처리

### CODE: 매칭 시킬 메타데이터 생성하기

In [5]:
import re

inquiry_type = {}

consulation = {
    "Quotation": "Quotation or Purchase Consultation",  # 견적 또는 구매 상담
    "Sales": "Quotation or Purchase Consultation",
    "Product": "Quotation or Purchase Consultation",
    "Purchase": "Quotation or Purchase Consultation",
    "Event": "Quotation or Purchase Consultation",
    "Partner": "Quotation or Purchase Consultation",
}

demo = {
    "Demo": "Request a Demo",  # 데모 요청하기
}

oem_odm = {
    "Oem": "OEM/ODM Request",
    "Odm": "OEM/ODM Request",
}

technic = {
    "Usage": "Usage or Technical Consultation",  # 사용 또는 기술 상담
    "Technical": "Usage or Technical Consultation",
    "Trainings": "Usage or Technical Consultation",
    "Services": "Usage or Technical Consultation",
    "Suggestions": "Usage or Technical Consultation",
}


distributorship = {
    "Distributorship": "Request for Distributorship",  # 대리점에 요청
}


others = {
    "Other": "Other",
    "Etc": "Other",
}

inquiry_category = [consulation, demo, oem_odm, technic, distributorship, others]

for category in inquiry_category:
    inquiry_type.update(category)

In [6]:
train["inquiry_type"].value_counts()

inquiry_type
Quotation or purchase consultation                                                                                     23274
Quotation or Purchase Consultation                                                                                     18766
Sales Inquiry                                                                                                           9977
Product Information                                                                                                     1237
-                                                                                                                        941
Other                                                                                                                    929
Usage or technical consultation                                                                                          668
Trainings                                                                                                       

In [7]:
start_patterns = [re.compile(f"^(?i){i}") for i in inquiry_type.keys()]
exists_patterns = [re.compile(f"(?i){i}") for i in inquiry_type.keys()]


def new_inquiry_type(old_inquiry_type):
    for pattern in start_patterns:
        find = pattern.search(old_inquiry_type.strip())
        if find:
#             print(f"find: {find.group().capitalize()}, old: {old_inquiry_type}")
            return inquiry_type.get(find.group().capitalize())
#     print(f"not found: {old_inquiry_type}")
    return "Other"


def retry_unknown_value_mapping(new_inquiry_type, old_inquiry_type):
    if new_inquiry_type == "Other":
        for pattern in exists_patterns:
            find = pattern.search(old_inquiry_type.strip())
            if find:
                return inquiry_type.get(find.group().capitalize())
        return "-"

    return new_inquiry_type

In [8]:
train["new_inquiry_type"] = train["inquiry_type"].apply(new_inquiry_type)
train["new_inquiry_type"] = train.apply(lambda row: retry_unknown_value_mapping(row["new_inquiry_type"], row["inquiry_type"]), axis=1)

### DATA: 매핑되지 않은 데이터 확인

In [9]:
train["new_inquiry_type"].value_counts()

new_inquiry_type
Quotation or Purchase Consultation    54009
Usage or Technical Consultation        2639
Other                                  1401
-                                       986
Request a Demo                          184
Request for Distributorship              75
OEM/ODM Request                           5
Name: count, dtype: int64

In [10]:
train[["inquiry_type", "new_inquiry_type"]][train["new_inquiry_type"] == "-"]["new_inquiry_type"].value_counts()

new_inquiry_type
-    986
Name: count, dtype: int64

#### 문의 유형을 똑바로 적지 않았지만 영업전환이 된 고객 157명

In [11]:
train[(train["new_inquiry_type"] == "-") & (train["is_converted"] == True)]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,new_inquiry_type
2232,0.75,/Guarulhos/Brazil,ID,NaN,25096,NaN,SMB,NaN,NaN,NaN,...,(select id_timeline),0,0,NaN,NaN,NaN,NaN,480,True,-
2455,0.25,NaN,ID,NaN,20091,NaN,Enterprise,53.0,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,487,True,-
2458,0.00,NaN,ID,NaN,25096,NaN,SMB,NaN,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,488,True,-
2461,0.25,NaN,ID,NaN,25096,NaN,Enterprise,8.0,NaN,NaN,...,september,0,0,NaN,NaN,NaN,NaN,167,True,-
2462,0.25,NaN,ID,NaN,25096,NaN,SMB,NaN,NaN,NaN,...,december 2022,0,0,NaN,NaN,NaN,NaN,487,True,-
2463,0.25,NaN,ID,NaN,25096,NaN,SMB,1.0,NaN,NaN,...,october 2022,0,0,NaN,NaN,NaN,NaN,167,True,-
2464,0.50,NaN,ID,NaN,25096,NaN,SMB,NaN,NaN,NaN,...,december 2022,0,0,NaN,NaN,NaN,NaN,167,True,-
2465,0.50,Araras / SP / BR,ID,NaN,25096,NaN,SMB,NaN,NaN,NaN,...,november 2022,0,0,NaN,NaN,NaN,NaN,279,True,-
13963,0.25,NaN,AS,NaN,25096,NaN,SMB,5.0,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,627,True,-
13964,0.25,NaN,AS,NaN,25096,NaN,SMB,NaN,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,625,True,-


### "-" 처리에 대한 방법들
1. "-"을 "Other"로 처리하자

In [12]:
train.loc[train["new_inquiry_type"] == "-", "new_inquiry_type"] = "Other"

In [13]:
train["new_inquiry_type"].value_counts()

new_inquiry_type
Quotation or Purchase Consultation    54009
Usage or Technical Consultation        2639
Other                                  2387
Request a Demo                          184
Request for Distributorship              75
OEM/ODM Request                           5
Name: count, dtype: int64

### lead description length 생략
- lead description길이가 inquiry type 의 길이와 상이함

## expected timeline 

### Category
- Timeline
- Less than 3 Months
- 3 Months ~ 6 Months
- 6 Months ~ 9 Months
- 9 Months ~ 1 Year
- More than a year

In [14]:
train["expected_timeline"].value_counts()

expected_timeline
less than 3 months                                                                                                                                                                                                                                                 17250
3 months ~ 6 months                                                                                                                                                                                                                                                 5026
more than a year                                                                                                                                                                                                                                                    3023
9 months ~ 1 year                                                                                                                                                                          

### Timeline data preprocess step-by-step
1. 카테고리와 정확히 매칭되는 데이터 추출하여 통일화
2. 카테고리와 내용은 같지만 특수문자 등으로 인해 1번에서 매칭되지 않은 데이터 통일화
3. less, greater 등 언어로 기간을 측정하는 데이터 카테고리화

#### Null 처리

In [15]:
train["expected_timeline"].isnull().sum()

30863

In [16]:
train["expected_timeline"].fillna("Space", inplace=True)

#### step 1 & 2: 카테고리와 매칭되는 데이터 + 매칭 되지만 특수문자가 포함 된 데이터 카테고리화

In [17]:
train[train["expected_timeline"].str.contains("~")]["expected_timeline"].value_counts()

expected_timeline
3 months ~ 6 months    5026
9 months ~ 1 year      1101
6 months ~ 9 months    1098
3_months_~_6_months       9
9_months_~_1_year         6
6_months_~_9_months       4
14:00~15:00               1
Name: count, dtype: int64

In [18]:
def convert_timeline_in_tilda(timeline):
    if "~" in timeline:
        timeline = timeline.replace("_", " ")
        split_timeline = timeline.split("~")
        return split_timeline[0].strip() + " ~ " + split_timeline[1].strip()
    return timeline

train["new_expected_timeline"] = train["expected_timeline"].apply(convert_timeline_in_tilda)

In [19]:
train[train["new_expected_timeline"].str.contains("~")]["new_expected_timeline"].value_counts()

new_expected_timeline
3 months ~ 6 months    5035
9 months ~ 1 year      1107
6 months ~ 9 months    1102
14:00 ~ 15:00             1
Name: count, dtype: int64

#### step 3: Less를 사용하여 날짜 범위를 지정한 경우 카테고리화 

In [20]:
train[train["new_expected_timeline"].str.contains("less")]["new_expected_timeline"].value_counts()

new_expected_timeline
less than 3 months                                                                                        17250
less than 6 months                                                                                          108
less_than_3_months                                                                                           76
less than 5 months                                                                                            2
less then 6 months                                                                                            2
less than 3 months- outdoor led requiment                                                                     1
less than 3 months ,meeting with the customer for the more details and tentative boq will ne 32 and 43        1
less than 3 months. customer not answered . to call back                                                      1
duplicate lead - il220100042906. less than 3 months                               

#### less

In [21]:
def less_value_categorial(timeline):
    find = None
    numeric_value = None
    scope = {
        "3": "Less than 3 Months",
        "6": "3 Months ~ 6 Months",
        "9": "6 Months ~ 9 Months",
    }
    
    numeric_pattern = re.compile("\d")  # NOTE: less 다음에 오는 숫자 데이터 추출
    if "less" in timeline:
        less_index = timeline.index("less")
        find = re.search(r'\d{1,2}', timeline[less_index:])
#         find = numeric_pattern.search(timeline)

    
    if find:
        numeric_value = find.group()
        
        for k,v in scope.items():
            if int(numeric_value) <= int(k):
                return v
    return timeline

In [22]:
train[train["new_expected_timeline"].str.contains("less")]["new_expected_timeline"].value_counts()

new_expected_timeline
less than 3 months                                                                                        17250
less than 6 months                                                                                          108
less_than_3_months                                                                                           76
less than 5 months                                                                                            2
less then 6 months                                                                                            2
less than 3 months- outdoor led requiment                                                                     1
less than 3 months ,meeting with the customer for the more details and tentative boq will ne 32 and 43        1
less than 3 months. customer not answered . to call back                                                      1
duplicate lead - il220100042906. less than 3 months                               

#### more

In [23]:
def more_value_categorial(timeline):
    find = None
    numeric_value = None
    scope = {
        "9": "More than a year",
        "6": "6 Months ~ 9 Months",
        "3": "3 Months ~ 6 Months",
        "0": "Less than 3 Months",
    }
    
    numeric_pattern = re.compile("\d")  # NOTE: less 문구의 숫자 데이터 추출
    if "more" in timeline:
        more_index = timeline.index("more")
        find = re.search(r'\d{1,2}', timeline[more_index:])

    
    if find:
        numeric_value = find.group()
        
        for k,v in scope.items():
            if int(numeric_value) >= int(k):
                return v
    return timeline.replace("_", " ")


In [24]:
train["new_expected_timeline"] = train["new_expected_timeline"].apply(less_value_categorial)
train["new_expected_timeline"] = train["new_expected_timeline"].apply(more_value_categorial)

In [25]:
train[train["new_expected_timeline"].str.contains("more")]["new_expected_timeline"].value_counts()

new_expected_timeline
more than a year                                           3027
update- 14th sept-- more details to be shared by client       1
Name: count, dtype: int64

In [26]:
train["new_expected_timeline"].value_counts()

new_expected_timeline
Space                                                                                                                                                                                                                                                              30863
Less than 3 Months                                                                                                                                                                                                                                                 17330
3 months ~ 6 months                                                                                                                                                                                                                                                 5035
more than a year                                                                                                                                                                       

### Follow up, No Response Cateogry, Budget Issue, Not Interest 추가 분류

In [27]:
expected_timeline_dict = {
    "Follow": "Follow up",
    "Already": "Follow up",
    "Respond": "No Response",
    "Response": "No Response",
    "Required": "No Response",
    "Requirement": "No Response",
    "Budget": "Budget Issue",
    "Interest": "Not Interest",
}


expected_timeline_start_patterns = [re.compile(f"^(?i){i}") for i in expected_timeline_dict.keys()]
expected_timeline_exists_patterns = [re.compile(f"(?i){i}") for i in expected_timeline_dict.keys()]
print(expected_timeline_exists_patterns)

# def new_expected_timeline(expected_timeline):
#     for pattern in expected_timeline_start_patterns:
#         find = pattern.search(expected_timeline.strip())
#         if find:
# #             print(f"find: {find.group().capitalize()}, old: {old_inquiry_type}")
#             return expected_timeline_dict.get(find.group().capitalize())
# #     print(f"not found: {old_inquiry_type}")
#     return expected_timeline


def timeline_retry_unknown_value_mapping(new_expected_timeline):
    for pattern in expected_timeline_exists_patterns:
        find = pattern.search(new_expected_timeline.strip())
        if find:
            return expected_timeline_dict.get(find.group().capitalize())

    return new_expected_timeline

[re.compile('(?i)Follow', re.IGNORECASE), re.compile('(?i)Already', re.IGNORECASE), re.compile('(?i)Respond', re.IGNORECASE), re.compile('(?i)Response', re.IGNORECASE), re.compile('(?i)Required', re.IGNORECASE), re.compile('(?i)Requirement', re.IGNORECASE), re.compile('(?i)Budget', re.IGNORECASE), re.compile('(?i)Interest', re.IGNORECASE)]


In [28]:
train["new_expected_timeline"].value_counts()

new_expected_timeline
Space                                                                                                                                                                                                                                                              30863
Less than 3 Months                                                                                                                                                                                                                                                 17330
3 months ~ 6 months                                                                                                                                                                                                                                                 5035
more than a year                                                                                                                                                                       

In [29]:
# train["new_expected_timeline"] = train["new_expected_timeline"].apply(new_expected_timeline)
train["new_expected_timeline"] = train["new_expected_timeline"].apply(timeline_retry_unknown_value_mapping)
# train["new_expected_timeline"] = train["new_expected_timeline"].apply(fill_unknown_value)

In [30]:
categories = ['Less than 3 Months', '3 months ~ 6 months', 'more than a year', '9 months ~ 1 year', '6 months ~ 9 months', 'Follow up', '3 Months ~ 6 Months', 'No Response', 'Budget Issue', 'Not Interest', "Space"]
train['new_expected_timeline'].where(train['new_expected_timeline'].isin(categories), 'Unknown').value_counts()

new_expected_timeline
Space                  30863
Less than 3 Months     17330
3 months ~ 6 months     5035
more than a year        3027
9 months ~ 1 year       1107
6 months ~ 9 months     1102
Unknown                  427
Follow up                144
3 Months ~ 6 Months      114
No Response              113
Budget Issue              29
Not Interest               8
Name: count, dtype: int64

In [42]:
train[(train["new_expected_timeline"]=="Space") & train["is_converted"] == True]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,new_inquiry_type,new_expected_timeline
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,...,1,0,0.003079,0.026846,corporate / office,Engineering,6,True,Quotation or Purchase Consultation,Space
13,0.50,/Benin City/Nigeria,AS,0.040816,30294,End-Customer,SMB,NaN,NaN,NaN,...,1,0,0.003079,0.026846,corporate / office,Construction,12,True,Quotation or Purchase Consultation,Space
14,0.25,/Lagos/Nigeria,AS,0.040816,16481,End-Customer,Enterprise,NaN,NaN,NaN,...,1,0,0.003079,0.026846,corporate / office,Construction,12,True,Quotation or Purchase Consultation,Space
17,0.50,/Singapore/Singapore,AS,0.066667,46362,Channel Partner,SMB,47.0,NaN,NaN,...,0,0,0.003079,0.026846,corporate / office,NaN,14,True,Quotation or Purchase Consultation,Space
396,0.75,//Saudi Arabia,ID,0.075000,33029,End-Customer,SMB,NaN,1.0,NaN,...,1,1,0.003079,0.064566,corporate / office,Others,153,True,Quotation or Purchase Consultation,Space
413,0.50,//India,ID,0.057534,26026,End-Customer,Enterprise,NaN,1.0,NaN,...,1,1,0.003079,0.064566,corporate / office,Engineering,165,True,Quotation or Purchase Consultation,Space
422,0.50,/Lodz /Poland,ID,0.019900,40621,End-Customer,SMB,NaN,1.0,NaN,...,1,0,0.003079,0.064566,corporate / office,Others,169,True,Quotation or Purchase Consultation,Space
430,0.50,//Saudi Arabia,ID,0.075000,25096,NaN,SMB,NaN,1.0,NaN,...,0,1,0.003079,0.064566,corporate / office,NaN,153,True,Quotation or Purchase Consultation,Space
1422,0.50,/Surat/India,ID,0.057534,10699,NaN,Enterprise,NaN,1.0,NaN,...,0,0,0.003079,0.064566,corporate / office,NaN,166,True,Quotation or Purchase Consultation,Space
1423,0.50,/Surat/India,ID,0.057534,10699,NaN,Enterprise,NaN,1.0,NaN,...,0,0,0.003079,0.064566,corporate / office,NaN,166,True,Quotation or Purchase Consultation,Space
